In [20]:
import pandas as pd
import scipy as sp
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.anova import anova_lm
import scipy.stats as stats

In [ ]:
#preprocessing
gene = pd.read_csv("~/.../EC_metagenome_out/pred_metagenome_strat.tsv",header=0,index_col=0,sep='\t')
sequence = gene.sequence
gene_relab = pd.read_csv("~/.../EC_metagenome_out/ecs_relab_strat_names.tsv",header=0,index_col=0,sep='\t')
gene_relab = pd.concat([sequence,gene_relab],axis=1)
scfa = ["EC:5.3.3.3","EC:4.2.1.17","EC:4.2.1.120","EC:2.8.3.1","EC:2.7.2.7","EC:2.7.2.1","EC:2.3.1.9","EC:2.3.1.19","EC:1.3.8.1",
       "EC:1.1.1.27","EC:1.2.1.10","EC:6.2.1.1","EC:4.2.1.55","EC:2.8.3.8","EC:2.7.2.15","EC:2.3.1.8","EC:1.3.1.44","EC:2.1.3.1"]

gene_scfa = gene_relab.loc[scfa].sort_index()
taxa=pd.read_csv("~/.../asv_genus.tsv",header=0,index_col=0,sep='\t')
taxonomy=taxa.taxonomy.to_dict()
gene_scfa_taxa = gene_scfa.replace(taxonomy)
gene_scfa_taxa_grouped=gene_scfa_taxa.groupby([gene_scfa_taxa.index,gene_scfa_taxa.sequence,gene_scfa_taxa.description]).sum()
gene_scfa_taxa_grouped.to_csv("~/.../gene_scfa_taxa.csv")

In [37]:
gene = pd.read_csv("~/.../gene_scfa_taxa.csv",index_col=[0,1])
gene_group = gene.transpose()

metadata = pd.read_csv("~/.../metadata_all.csv",index_col=0)
gene_sample = metadata.index.intersection(gene_group.index)
gene_metadata = pd.concat([metadata.loc[gene_sample],gene_group.loc[gene_sample]],sort = True, axis =1 )
gene_name=gene.description

In [22]:
#for a single time point only
G30 = (metadata.Group=='Y_30_Fecal') | (metadata.Group=='A_30_Fecal')
gene_metadata_30 = gene_metadata[G30]

G60 = (gene_metadata.Group=='Y_60_Fecal') | (gene_metadata.Group=='A_60_Fecal')
gene_metadata_60 = gene_metadata[G60]

G90 = (gene_metadata.Group=='Y_90_Fecal') | (gene_metadata.Group=='A_90_Fecal')
gene_metadata_90 = gene_metadata[G90]

In [42]:
def gene_stat(result):
    #there are a ton of stuff can be simplified or optimized for more general use: I can load the file and filter groups based on given names
    #i can set an argument to determine the numbers of groups, factors and determine which stats should be run
    res_sca=result  
    genes = res_sca.columns.tolist()[4:]
    res_stat = pd.DataFrame()
    res_mc = pd.DataFrame()

    for gene in genes: 
        test = res_sca.loc[:,["Group", "Y-A","Time","Location",gene]]
        test.columns = ["Group","YA","time","site","value"]
        test.loc[:,"value"]=test.loc[:,"value"].astype(float)
        model = ols('value ~ C(YA)*C(time)*C(site)', test).fit()
        aov_table = anova_lm(model, typ=1)
        res_stat=res_stat.append(aov_table["PR(>F)"].transpose()[0:7])
        mc = statsmodels.stats.multicomp.MultiComparison(test['value'],test['Group'])
        mc_BH_table = mc.allpairtest(stats.ttest_ind,method='fdr_bh')[0]
        mc_table=pd.DataFrame(mc_BH_table.data[1:],columns=mc_BH_table.data[0])
        res_mc = res_mc.append(mc_table["pval"].transpose())
    res_mc.index = genes
    res_stat.index = genes
    res_mc.columns = mc_table["group1"]+"_"+mc_table["group2"]
    res_stats = pd.concat([res_stat,res_mc],axis =1)
    return res_stats #,res_sca_sig

In [44]:
#change gene_metadata to gene_metadata_*timepoint for one timepoint analysis
gene_abun_stat = gene_stat(gene_metadata)
gene_abun_stat_name = pd.concat([gene_name,gene_abun_stat],axis = 1)
gene_abun_stat_name[['description',"A_30_Fecal_Y_30_Fecal","A_60_Fecal_Y_60_Fecal","A_90_Fecal_Y_90_Fecal"]].to_csv('~/.../scfa_enzymes_taxa_stats.csv')